In [1]:
%load_ext autoreload
%autoreload 2

from qiskit_metal import MetalGUI, designs
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.renderers.renderer_gmsh.gmsh_renderer import QGmshRenderer
from qiskit_metal.qlibrary.qubits.transmon_pocket_cl import TransmonPocketCL
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.anchored_path import RouteAnchors
from qiskit_metal.qlibrary.tlines.pathfinder import RoutePathfinder
from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.renderers.renderer_elmer.elmer_renderer import QElmerRenderer
from qiskit_metal.renderers.renderer_elmer.elmer_renderer import load_capacitance_matrix_from_file
import numpy as np
from collections import OrderedDict
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings


In [2]:
design = designs.MultiPlanar({}, True)
gui = MetalGUI(design)
design.overwrite_enabled = True

In [ ]:
conn_pads = dict(
    connection_pads = dict(
        readout = dict(loc_W=0, loc_H=-1),
        coupler1 = dict(loc_W=-1, loc_H=1),
        coupler2 = dict(loc_W=1, loc_H=1)
    )
)


q1 = TransmonPocketCL(design, 'Q1', options = dict(
    pos_x='+1.1mm', pos_y='+0.8mm', orientation = '0', **transmon_options))
q2 = TransmonPocketCL(design, 'Q2', options = dict(
    pos_x='+2.7mm', pos_y='+0.8mm', orientation = '0', **transmon_options))
q3 = TransmonPocketCL(design, 'Q3', options = dict(
    pos_x='+2.7mm', pos_y='-0.3mm', orientation = '0', **transmon_options))
q4 = TransmonPocketCL(design, 'Q4', options = dict(
    pos_x='+0.0mm', pos_y='-0.7mm', orientation = '0', **transmon_options))
q5 = TransmonPocketCL(design, 'Q5', options = dict(
    pos_x='-2.7mm', pos_y='-0.3mm', orientation = '0', **transmon_options))
q6 = TransmonPocketCL(design, 'Q6', options = dict(
    pos_x='-1.1mm', pos_y='+0.8mm', orientation = '0', **transmon_options))
q7 = TransmonPocketCL(design, 'Q7', options = dict(
    pos_x='-2.7mm', pos_y='+0.8mm', orientation = '0', **transmon_options))

offset_tm = 69  

fillet='99.99um'
cpw_options = Dict(
    lead=Dict(
        start_straight='100um',
        end_straight='250um'),
    fillet=fillet
    )

def connect(cpw_name: str, pin1_comp_name: str, pin1_comp_pin: str, pin2_comp_name: str, pin2_comp_pin: str,
            length: str, asymmetry='0 um'):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        pin_inputs=Dict(
            start_pin=Dict(
                component=pin1_comp_name,
                pin=pin1_comp_pin),
            end_pin=Dict(
                component=pin2_comp_name,
                pin=pin2_comp_pin)),
        total_length=length)
    myoptions.update(cpw_options)
    myoptions.meander.asymmetry = asymmetry
    return RouteMeander(design, cpw_name, myoptions) 

asym = 500
cpw1 = connect('cpw1', 'Q7', 'b', 'Q5', 'c', '50um', f'-{asym-1.25*offset_tm}um')
cpw3 = connect('cpw3', 'Q7', 'a', 'Q6', 'b', '50um', f'-{asym-1.25*offset_tm}um')
cpw4 = connect('cpw4', 'Q6', 'a', 'Q1', 'b', '50um', f'-{asym-1.25*offset_tm}um')
cpw5 = connect('cpw5', 'Q5', 'd', 'Q4', 'c', '50um', f'-{asym-1.25*offset_tm}um')
cpw6 = connect('cpw6', 'Q4', 'd', 'Q3', 'c', '50um', f'-{asym-1.25*offset_tm}um')
cpw7 = connect('cpw7', 'Q1', 'a', 'Q2', 'b', '50um', f'-{asym-1.25*offset_tm}um')
cpw8 = connect('cpw8', 'Q2', 'a', 'Q3', 'd', '50um', f'-{asym-1.25*offset_tm}um')


p1_q = LaunchpadWirebondCoupled(design, 'P1_Q', options = dict(pos_x='-4.25mm', pos_y='+3.0mm', orientation='270', lead_length='30um'))
p2_q = LaunchpadWirebondCoupled(design, 'P2_Q', options = dict(pos_x='-1.5mm', pos_y='+3.0mm', orientation='270', lead_length='30um'))
p3_q = LaunchpadWirebondCoupled(design, 'P3_Q', options = dict(pos_x='+1.0mm', pos_y='+3.0mm', orientation='270', lead_length='30um'))
p4_q = LaunchpadWirebondCoupled(design, 'P4_Q', options = dict(pos_x='+2.5mm', pos_y='+3.0mm', orientation='270', lead_length='30um'))
p5_q = LaunchpadWirebondCoupled(design, 'P5_Q', options = dict(pos_x='-4.25mm', pos_y='-2.5mm', orientation='90', lead_length='30um'))
p6_q = LaunchpadWirebondCoupled(design, 'P6_Q', options = dict(pos_x='-1.4mm', pos_y='-2.5mm', orientation='90', lead_length='30um'))
p7_q = LaunchpadWirebondCoupled(design, 'P7_Q', options = dict(pos_x='+2.5mm', pos_y='-2.5mm', orientation='90', lead_length='30um'))

gui.rebuild()
gui.autoscale()

fillet = '99.99um'

def connect(cpw_name: str, pin1_comp_name: str, pin1_comp_pin: str, pin2_comp_name: str, pin2_comp_pin: str,
            length: str, asymmetry='0 um'):
    myoptions = Dict(
        pin_inputs=Dict(
            start_pin=Dict(component=pin1_comp_name, pin=pin1_comp_pin),
            end_pin=Dict(component=pin2_comp_name, pin=pin2_comp_pin)),
        total_length=length)
    myoptions.update(cpw_options)
    myoptions.meander.asymmetry = asymmetry
    return RouteMeander(design, cpw_name, myoptions)

cpw_options = Dict(
    lead=Dict(
        start_straight='430um',
        end_straight='0um'),
    fillet=fillet
)
asym = 150
ol1 = connect('ol1', 'Q7', 'c', 'P1_Q', 'tie', '8.6 mm', f'+{asym}um')
ol3 = connect('ol3', 'Q1', 'd', 'P3_Q', 'tie', '8.6 mm', f'+{asym}um')

cpw_options = Dict(
    lead=Dict(
        start_straight='535um',
        end_straight='0um'),
    fillet=fillet
)
asym = 200
ol2 = connect('ol2', 'Q6', 'd', 'P2_Q', 'tie', '8.6 mm', f'+{asym}um')
ol4 = connect('ol4', 'Q2', 'd', 'P4_Q', 'tie', '8.6 mm', f'+{asym}um')

cpw_options = Dict(
    lead=Dict(
        start_straight='480um',
        end_straight='0um'),
    fillet=fillet
)
asym = 180
ol5 = connect('ol5', 'Q5', 'b', 'P5_Q', 'tie', '8.6 mm', f'+{asym}um')
ol6 = connect('ol6', 'Q4', 'b', 'P6_Q', 'tie', '8.6 mm', f'+{asym}um')
ol7 = connect('ol7', 'Q3', 'a', 'P7_Q', 'tie', '8.6 mm', f'+{asym}um')

gui.rebuild()
gui.autoscale()
#gui.highlight_components(['Q1','Q2','Q3','Q4','Q5','Q6','Q7'])


11:15PM 43s WARNING [check_lengths]: For path table, component=cpw4, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
11:15PM 43s WARNING [check_lengths]: For path table, component=cpw4, key=cut has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
11:15PM 43s WARNING [check_lengths]: For path table, component=cpw5, key=trace has short segments that could cause issues with fillet. Values in (15-18)  are index(es) in shapely geometry.
11:15PM 43s WARNING [check_lengths]: For path table, component=cpw5, key=cut has short segments that could cause issues with fillet. Values in (15-18)  are index(es) in shapely geometry.
11:15PM 45s WARNING [check_lengths]: For path table, component=cpw4, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
11:15PM 45s WARNING [check_lengths]: For path table, component=cp

In [4]:
#gmsh_renderer = QGmshRenderer(design)
#gmsh_renderer.render_design(open_pins=[("Q1", "a"), ("Q1", "b"), ("Q1", "c"), ("Q1", "d"),
                                        #("Q2", "a"), ("Q2", "b"), ("Q2", "c"), ("Q2", "d"),
                                        #("Q3", "a"), ("Q3", "b"), ("Q3", "c"), ("Q3", "d"),
                                        #("Q4", "a"), ("Q4", "b"), ("Q4", "c"), ("Q4", "d"),
                                        #("Q5", "a"), ("Q5", "b"), ("Q5", "c"), ("Q5", "d"),
                                        #("Q6", "a"), ("Q6", "b"), ("Q6", "c"), ("Q6", "d"),
                                        #("Q7", "a"), ("Q7", "b"), ("Q7", "c"), ("Q7", "d")],
                                        #mesh_geoms=False)


In [5]:
#gmsh_renderer.launch_gui()

In [6]:
#gmsh_renderer.add_mesh(dim=3, intelli_mesh=False)
#gmsh_renderer.launch_gui()

In [7]:
#gmsh_renderer.options.mesh.min_size = '5um'
#gmsh_renderer.options.mesh.max_size = '20um'

#gmsh_renderer.render_design(open_pins=[("Q1", "a"), ("Q1", "b"), ("Q1", "c"), ("Q1", "d"),
                                        #("Q2", "a"), ("Q2", "b"), ("Q2", "c"), ("Q2", "d"),
                                        #("Q3", "a"), ("Q3", "b"), ("Q3", "c"), ("Q3", "d"),
                                        #("Q4", "a"), ("Q4", "b"), ("Q4", "c"), ("Q4", "d"),
                                        #("Q5", "a"), ("Q5", "b"), ("Q5", "c"), ("Q5", "d"),
                                        #("Q6", "a"), ("Q6", "b"), ("Q6", "c"), ("Q6", "d"),
                                        #("Q7", "a"), ("Q7", "b"), ("Q7", "c"), ("Q7", "d")],
                                         #mesh_geoms=False)

#gmsh_renderer.add_mesh(intelli_mesh=False)
#gmsh_renderer.launch_gui()

In [8]:
#gmsh_renderer.options.mesh.min_size = '5um'
#gmsh_renderer.options.mesh.max_size = '50um'

#gmsh_renderer.render_design(open_pins=[("Q1", "a"), ("Q1", "b"), ("Q1", "c"), ("Q1", "d"),
# ("Q2", "a"), ("Q2", "b"), ("Q2", "c"), ("Q2", "d"),("Q3", "a"), ("Q3", "b"), ("Q3", "c"), ("Q3", "d"),("Q4", "a"), ("Q4", "b"), ("Q4", "c"), ("Q4", "d"),
# ("Q5", "a"), ("Q5", "b"), ("Q5", "c"), ("Q5", "d"),
# ("Q6", "a"), ("Q6", "b"), ("Q6", "c"), ("Q6", "d"),
# ("Q7", "a"), ("Q7", "b"), ("Q7", "c"), ("Q7", "d")],
# mesh_geoms=True)
#gmsh_renderer.launch_gui()

In [9]:
#gmsh_renderer.export_mesh("test.msh")

In [10]:
#gmsh_renderer.close()

In [4]:
elmer_renderer = QElmerRenderer(design)
elmer_renderer.gmsh.options.mesh.min_size = "5um"
elmer_renderer.gmsh.options.mesh.max_size = "50um"

In [5]:
elmer_renderer.render_design(open_pins=[("Q1", "a"), ("Q1", "b"), ("Q1", "c"), ("Q1", "d"),
                                        ("Q2", "a"), ("Q2", "b"), ("Q2", "c"), ("Q2", "d"),
                                        ("Q3", "a"), ("Q3", "b"), ("Q3", "c"), ("Q3", "d"),
                                        ("Q4", "a"), ("Q4", "b"), ("Q4", "c"), ("Q4", "d"),
                                        ("Q5", "a"), ("Q5", "b"), ("Q5", "c"), ("Q5", "d"),
                                        ("Q6", "a"), ("Q6", "b"), ("Q6", "c"), ("Q6", "d"),
                                        ("Q7", "a"), ("Q7", "b"), ("Q7", "c"), ("Q7", "d")],
                                         mesh_geoms=True)

In [13]:
#elmer_renderer.launch_gmsh_gui()

In [14]:
#elmer_renderer.export_mesh()

In [6]:
elmer_renderer.add_solution_setup('capacitance')

In [7]:
elmer_renderer.run('capacitance')

11:07PM 46s INFO [run]: Running ElmerGrid on input mesh from Gmsh...
11:07PM 54s INFO [run]: Running ElmerSolver for solver type: 'capacitance'


TypeError: can't multiply sequence by non-int of type 'float'

In [ ]:
elmer_renderer.capacitance_matrix

,cpw7_trace,cpw5_trace,cpw1_trace,ol5_trace,Q5_a_wire,Q5_pad_bot,Q5_pad_top,cpw6_trace,ol6_trace,Q4_a_wire,...,Q7_pad_top,Q6_c_wire,Q2_c_wire,P7_Q_launch_pad,Q2_pad_bot,Q1_c_connector_pad,Q2_pad_top,Q1_pad_bot,Q1_pad_top,ground_plane
cpw7_trace,4.122719e+02,-0.002155,-0.000039,-3.455594e-05,-1.937136e-05,-0.000065,-0.000068,-0.067675,-0.002430,-0.001437,...,-0.000082,-0.000319,-2.818812e-01,-2.289763e-04,-1.606670e+01,-6.997695e-02,-1.521200e+00,-10.832813,-1.244107,-380.625546
cpw5_trace,-2.155476e-03,984.209850,-0.274896,-1.488274e-01,-2.917387e-01,-1.527656,-14.604287,-0.249024,-0.559628,-0.060308,...,-0.053286,-0.010370,-1.414719e-04,-1.124141e-04,-4.654239e-04,-1.937689e-03,-4.109172e-04,-0.007817,-0.006243,-950.005975
cpw1_trace,-3.864892e-05,-0.274896,658.235041,-4.306013e-01,-6.693921e-02,-1.445801,-14.640564,-0.000328,-0.004404,-0.000140,...,-1.547413,-0.004262,-3.155486e-06,-1.567933e-06,-8.822154e-06,-5.631448e-05,-8.594678e-06,-0.000157,-0.000152,-622.899201
ol5_trace,-3.455594e-05,-0.148827,-0.430601,2.191203e+03,-2.068203e-01,-16.268823,-1.641417,-0.000561,-0.017113,-0.000281,...,-0.034092,-0.001543,-2.761006e-06,-6.025811e-06,-8.612685e-06,-4.005627e-05,-7.908231e-06,-0.000129,-0.000116,-2151.176968
Q5_a_wire,-1.937136e-05,-0.291739,-0.066939,-2.068203e-01,4.309813e+01,-10.540596,-1.041510,-0.000281,-0.005647,-0.000133,...,-0.012381,-0.001031,-1.465006e-06,-1.509117e-06,-4.428117e-06,-2.416661e-05,-4.122707e-06,-0.000076,-0.000069,-30.880353
Q5_pad_bot,-6.474881e-05,-1.527656,-1.445801,-1.626882e+01,-1.054060e+01,107.720533,-36.804820,-0.000859,-0.016263,-0.000400,...,-0.072374,-0.004034,-4.963819e-06,-4.681634e-06,-1.481384e-05,-8.299644e-05,-1.389423e-05,-0.000255,-0.000233,-40.780437
Q5_pad_top,-6.769159e-05,-14.604287,-14.640564,-1.641417e+00,-1.041510e+00,-36.804820,109.282716,-0.000814,-0.013607,-0.000369,...,-0.105444,-0.004957,-5.204901e-06,-4.075333e-06,-1.530422e-05,-8.976146e-05,-1.446868e-05,-0.000270,-0.000250,-40.063806
cpw6_trace,-6.767497e-02,-0.249024,-0.000328,-5.614553e-04,-2.814174e-04,-0.000859,-0.000814,1309.509356,-0.168423,-0.282256,...,-0.000437,-0.001091,-1.259677e-02,-3.950537e-03,-7.995860e-02,-7.353849e-03,-5.289782e-02,-0.047051,-0.034797,-1275.349926
ol6_trace,-2.430276e-03,-0.559628,-0.004404,-1.711263e-02,-5.647486e-03,-0.016263,-0.013607,-0.168423,2190.725628,-0.226868,...,-0.003210,-0.003011,-1.860533e-04,-7.713355e-04,-6.769560e-04,-1.664919e-03,-5.731215e-04,-0.007496,-0.005785,-2150.208013
Q4_a_wire,-1.436882e-03,-0.060308,-0.000140,-2.809021e-04,-1.334890e-04,-0.000400,-0.000369,-0.282256,-0.226868,42.848269,...,-0.000163,-0.000331,-1.072883e-04,-1.681498e-04,-3.958416e-04,-8.047216e-04,-3.312496e-04,-0.004107,-0.003026,-30.644257


In [18]:
#elmer_renderer.display_post_processing_data()

In [ ]:
elmer_renderer.save_capacitance_matrix("cap_matrix.txt2")

elmer_renderer.close()

In [127]:
import numpy as np
from qiskit_metal.analyses.quantization.lom_core_analysis import CompositeSystem, Cell, Subsystem 
from scipy.constants import speed_of_light as c_light
import matplotlib.pyplot as plt
%matplotlib inline

In [143]:
cap_matrix1 = load_capacitance_matrix_from_file("cap_matrix.txt2")

In [50]:
opt1 = {
    'node_rename': {
        'Q1_a': 'a1', 'Q1_b': 'b1', 'Q1_c': 'c1', 'Q1_d': 'd1',
        'Q2_a': 'a2', 'Q2_b': 'b2', 'Q2_c': 'c2', 'Q2_d': 'd2',
        'Q3_a': 'a3', 'Q3_b': 'b3', 'Q3_c': 'c3', 'Q3_d': 'd3',
        'Q4_a': 'a4', 'Q4_b': 'b4', 'Q4_c': 'c4', 'Q4_d': 'd4',
        'Q5_a': 'a5', 'Q5_b': 'b5', 'Q5_c': 'c5', 'Q5_d': 'd5',
        'Q6_a': 'a6', 'Q6_b': 'b6', 'Q6_c': 'c6', 'Q6_d': 'd6',
        'Q7_a': 'a7', 'Q7_b': 'b7', 'Q7_c': 'c7', 'Q7_d': 'd7',
        'GND': 'ground_plane',
    },
    'cap_mat': cap_matrix,
    'ind_dict': {
        ('Q1_pad_top', 'Q1_pad_bot'): 12,
        ('Q2_pad_top', 'Q2_pad_bot'): 12,
        ('Q3_pad_top', 'Q3_pad_bot'): 12,
        ('Q4_pad_top', 'Q4_pad_bot'): 12,
        ('Q5_pad_top', 'Q5_pad_bot'): 12,
        ('Q6_pad_top', 'Q6_pad_bot'): 12,
        ('Q7_pad_top', 'Q7_pad_bot'): 12
    },
    'jj_dict': {
        ('Q1_pad_top', 'Q1_pad_bot'): 'j1',
        ('Q2_pad_top', 'Q2_pad_bot'): 'j2',
        ('Q3_pad_top', 'Q3_pad_bot'): 'j3',
        ('Q4_pad_top', 'Q4_pad_bot'): 'j4',
        ('Q5_pad_top', 'Q5_pad_bot'): 'j5',
        ('Q6_pad_top', 'Q6_pad_bot'): 'j6',
        ('Q7_pad_top', 'Q7_pad_bot'): 'j7'
    },
    'cj_dict': {
        ('Q1_pad_top', 'Q1_pad_bot'): 2,
        ('Q2_pad_top', 'Q2_pad_bot'): 2,
        ('Q3_pad_top', 'Q3_pad_bot'): 2,
        ('Q4_pad_top', 'Q4_pad_bot'): 2,
        ('Q5_pad_top', 'Q5_pad_bot'): 2,
        ('Q6_pad_top', 'Q6_pad_bot'): 2,
        ('Q7_pad_top', 'Q7_pad_bot'): 2
    }
}

cell_1 = Cell(opt1)


In [51]:
# Q1
transmon1 = Subsystem(name='transmon1_sys', sys_type='TRANSMON', nodes=['j'])
coup1_1 = Subsystem(name='coup1_1_sys', sys_type='TL_RESONATOR', nodes=['a'], q_opts=dict(f_res=7.4, Z0=50, vp=0.404314 * c_light))
coup2_1 = Subsystem(name='coup1_2_sys', sys_type='TL_RESONATOR', nodes=['b'], q_opts=dict(f_res=7.2, Z0=50, vp=0.404314 * c_light))
readout1 = Subsystem(name='readout1_sys', sys_type='TL_RESONATOR', nodes=['c'], q_opts=dict(f_res=7.0, Z0=50, vp=0.404314 * c_light))
readout8 = Subsystem(name='readout8_sys', sys_type='TL_RESONATOR', nodes=['d'], q_opts=dict(f_res=6.8, Z0=50, vp=0.404314 * c_light))

# Q2
transmon2 = Subsystem(name='transmon2_sys', sys_type='TRANSMON', nodes=['j2'])
coup1_2 = Subsystem(name='coup2_1_sys', sys_type='TL_RESONATOR', nodes=['a2'], q_opts=dict(f_res=7.4, Z0=50, vp=0.404314 * c_light))
coup2_2 = Subsystem(name='coup2_2_sys', sys_type='TL_RESONATOR', nodes=['b2'], q_opts=dict(f_res=7.2, Z0=50, vp=0.404314 * c_light))
readout2 = Subsystem(name='readout2_sys', sys_type='TL_RESONATOR', nodes=['c2'], q_opts=dict(f_res=7.0, Z0=50, vp=0.404314 * c_light))
readout9 = Subsystem(name='readout9_sys', sys_type='TL_RESONATOR', nodes=['d2'], q_opts=dict(f_res=6.8, Z0=50, vp=0.404314 * c_light))

# Q3
transmon3 = Subsystem(name='transmon3_sys', sys_type='TRANSMON', nodes=['j3'])
coup1_3 = Subsystem(name='coup3_1_sys', sys_type='TL_RESONATOR', nodes=['a3'], q_opts=dict(f_res=7.4, Z0=50, vp=0.404314 * c_light))
coup2_3 = Subsystem(name='coup3_2_sys', sys_type='TL_RESONATOR', nodes=['b3'], q_opts=dict(f_res=7.2, Z0=50, vp=0.404314 * c_light))
readout3 = Subsystem(name='readout3_sys', sys_type='TL_RESONATOR', nodes=['c3'], q_opts=dict(f_res=7.2, Z0=50, vp=0.404314 * c_light))
readout10 = Subsystem(name='readout10_sys', sys_type='TL_RESONATOR', nodes=['d3'], q_opts=dict(f_res=6.8, Z0=50, vp=0.404314 * c_light))

# Q4
transmon4 = Subsystem(name='transmon4_sys', sys_type='TRANSMON', nodes=['j4'])
coup1_4 = Subsystem(name='coup4_1_sys', sys_type='TL_RESONATOR', nodes=['a4'], q_opts=dict(f_res=7.4, Z0=50, vp=0.404314 * c_light))
coup2_4 = Subsystem(name='coup4_2_sys', sys_type='TL_RESONATOR', nodes=['b4'], q_opts=dict(f_res=7.2, Z0=50, vp=0.404314 * c_light))
readout4 = Subsystem(name='readout4_sys', sys_type='TL_RESONATOR', nodes=['c4'], q_opts=dict(f_res=7.0, Z0=50, vp=0.404314 * c_light))
readout11 = Subsystem(name='readout11_sys', sys_type='TL_RESONATOR', nodes=['d4'], q_opts=dict(f_res=6.8, Z0=50, vp=0.404314 * c_light))

# Q5
transmon5 = Subsystem(name='transmon5_sys', sys_type='TRANSMON', nodes=['j5'])
coup1_5 = Subsystem(name='coup5_1_sys', sys_type='TL_RESONATOR', nodes=['a5'], q_opts=dict(f_res=7.4, Z0=50, vp=0.404314 * c_light))
coup2_5 = Subsystem(name='coup5_2_sys', sys_type='TL_RESONATOR', nodes=['b5'], q_opts=dict(f_res=7.2, Z0=50, vp=0.404314 * c_light))
readout5 = Subsystem(name='readout5_sys', sys_type='TL_RESONATOR', nodes=['c5'], q_opts=dict(f_res=7.0, Z0=50, vp=0.404314 * c_light))
readout12 = Subsystem(name='readout12_sys', sys_type='TL_RESONATOR', nodes=['d5'], q_opts=dict(f_res=6.8, Z0=50, vp=0.404314 * c_light))

# Q6
transmon6 = Subsystem(name='transmon6_sys', sys_type='TRANSMON', nodes=['j6'])
coup1_6 = Subsystem(name='coup6_1_sys', sys_type='TL_RESONATOR', nodes=['a6'], q_opts=dict(f_res=7.4, Z0=50, vp=0.404314 * c_light))
coup2_6 = Subsystem(name='coup6_2_sys', sys_type='TL_RESONATOR', nodes=['b6'], q_opts=dict(f_res=7.2, Z0=50, vp=0.404314 * c_light))
readout6 = Subsystem(name='readout6_sys', sys_type='TL_RESONATOR', nodes=['c6'], q_opts=dict(f_res=7.0, Z0=50, vp=0.404314 * c_light))
readout13 = Subsystem(name='readout13_sys', sys_type='TL_RESONATOR', nodes=['d6'], q_opts=dict(f_res=6.8, Z0=50, vp=0.404314 * c_light))

# Q7
transmon7 = Subsystem(name='transmon7_sys', sys_type='TRANSMON', nodes=['j7'])
coup1_7 = Subsystem(name='coup7_1_sys', sys_type='TL_RESONATOR', nodes=['a7'], q_opts=dict(f_res=7.4, Z0=50, vp=0.404314 * c_light))
coup2_7 = Subsystem(name='coup7_2_sys', sys_type='TL_RESONATOR', nodes=['b7'], q_opts=dict(f_res=7.2, Z0=50, vp=0.404314 * c_light))
readout7 = Subsystem(name='readout7_sys', sys_type='TL_RESONATOR', nodes=['c7'], q_opts=dict(f_res=7.0, Z0=50, vp=0.404314 * c_light))
readout14 = Subsystem(name='readout14_sys', sys_type='TL_RESONATOR', nodes=['d7'], q_opts=dict(f_res=6.8, Z0=50, vp=0.404314 * c_light))


In [52]:
composite_sys = CompositeSystem(
    subsystems=[
        transmon1, transmon2, transmon3, transmon4, transmon5, transmon6, transmon7,
        coup1_1, coup1_2, coup1_3, coup1_4, coup1_5, coup1_6, coup1_7,
        coup2_1, coup2_2, coup2_3, coup2_4, coup2_5, coup2_6, coup2_7,
        readout1, readout2, readout3, readout4, readout5, readout6, readout7,
        readout8, readout9, readout10, readout11, readout12, readout13, readout14
    ], 
    cells=[cell_1], 
    grd_node='ground_plane',
 nodes_force_keep = ['j', 'j2', 'j3', 'j4', 'j5', 'j6', 'j7']
)




In [53]:
print("All nodes in subsystems:")
for s in composite_sys.subsystems:
    print(f"{s.name}: {s.nodes}")


All nodes in subsystems:
transmon1_sys: ['j']
transmon2_sys: ['j2']
transmon3_sys: ['j3']
transmon4_sys: ['j4']
transmon5_sys: ['j5']
transmon6_sys: ['j6']
transmon7_sys: ['j7']
coup1_1_sys: ['a']
coup2_1_sys: ['a2']
coup3_1_sys: ['a3']
coup4_1_sys: ['a4']
coup5_1_sys: ['a5']
coup6_1_sys: ['a6']
coup7_1_sys: ['a7']
coup1_2_sys: ['b']
coup2_2_sys: ['b2']
coup3_2_sys: ['b3']
coup4_2_sys: ['b4']
coup5_2_sys: ['b5']
coup6_2_sys: ['b6']
coup7_2_sys: ['b7']
readout1_sys: ['c']
readout2_sys: ['c2']
readout3_sys: ['c3']
readout4_sys: ['c4']
readout5_sys: ['c5']
readout6_sys: ['c6']
readout7_sys: ['c7']
readout8_sys: ['d']
readout9_sys: ['d2']
readout10_sys: ['d3']
readout11_sys: ['d4']
readout12_sys: ['d5']
readout13_sys: ['d6']
readout14_sys: ['d7']


In [79]:
import numpy as np
from qiskit_metal.analyses.quantization.lom_core_analysis import CompositeSystem, Cell, Subsystem
from scipy.constants import speed_of_light as c_light
import matplotlib.pyplot as plt

%matplotlib inline

# Make sure you have your function to load capacitance matrix:
# cap_matrix = load_capacitance_matrix_from_file("cap_matrix.txt2")
# For now, I'll assume cap_matrix is already loaded as 'cap_matrix'.

opt1 = {
    'node_rename': {
        'Q1_a': 'a1', 'Q1_b': 'b1', 'Q1_c': 'c1', 'Q1_d': 'd1',
        'Q2_a': 'a2', 'Q2_b': 'b2', 'Q2_c': 'c2', 'Q2_d': 'd2',
        'Q3_a': 'a3', 'Q3_b': 'b3', 'Q3_c': 'c3', 'Q3_d': 'd3',
        'Q4_a': 'a4', 'Q4_b': 'b4', 'Q4_c': 'c4', 'Q4_d': 'd4',
        'Q5_a': 'a5', 'Q5_b': 'b5', 'Q5_c': 'c5', 'Q5_d': 'd5',
        'Q6_a': 'a6', 'Q6_b': 'b6', 'Q6_c': 'c6', 'Q6_d': 'd6',
        'Q7_a': 'a7', 'Q7_b': 'b7', 'Q7_c': 'c7', 'Q7_d': 'd7',
        'GND': 'ground_plane',
    },
    'cap_mat': cap_matrix,
    'ind_dict': {
        ('Q1_pad_top', 'Q1_pad_bot'): 12,
        ('Q2_pad_top', 'Q2_pad_bot'): 12,
        ('Q3_pad_top', 'Q3_pad_bot'): 12,
        ('Q4_pad_top', 'Q4_pad_bot'): 12,
        ('Q5_pad_top', 'Q5_pad_bot'): 12,
        ('Q6_pad_top', 'Q6_pad_bot'): 12,
        ('Q7_pad_top', 'Q7_pad_bot'): 12
    },
    'jj_dict': {
        ('Q1_pad_top', 'Q1_pad_bot'): 'j1',
        ('Q2_pad_top', 'Q2_pad_bot'): 'j2',
        ('Q3_pad_top', 'Q3_pad_bot'): 'j3',
        ('Q4_pad_top', 'Q4_pad_bot'): 'j4',
        ('Q5_pad_top', 'Q5_pad_bot'): 'j5',
        ('Q6_pad_top', 'Q6_pad_bot'): 'j6',
        ('Q7_pad_top', 'Q7_pad_bot'): 'j7'
    },
    'cj_dict': {
        ('Q1_pad_top', 'Q1_pad_bot'): 2,
        ('Q2_pad_top', 'Q2_pad_bot'): 2,
        ('Q3_pad_top', 'Q3_pad_bot'): 2,
        ('Q4_pad_top', 'Q4_pad_bot'): 2,
        ('Q5_pad_top', 'Q5_pad_bot'): 2,
        ('Q6_pad_top', 'Q6_pad_bot'): 2,
        ('Q7_pad_top', 'Q7_pad_bot'): 2
    }
}

cell_1 = Cell(opt1)

# Fix: Subsystem node names must match junction nodes in jj_dict
transmon1 = Subsystem(name='transmon1_sys', sys_type='TRANSMON', nodes=['j1'])
transmon2 = Subsystem(name='transmon2_sys', sys_type='TRANSMON', nodes=['j2'])
transmon3 = Subsystem(name='transmon3_sys', sys_type='TRANSMON', nodes=['j3'])
transmon4 = Subsystem(name='transmon4_sys', sys_type='TRANSMON', nodes=['j4'])
transmon5 = Subsystem(name='transmon5_sys', sys_type='TRANSMON', nodes=['j5'])
transmon6 = Subsystem(name='transmon6_sys', sys_type='TRANSMON', nodes=['j6'])
transmon7 = Subsystem(name='transmon7_sys', sys_type='TRANSMON', nodes=['j7'])

# Couplers and readouts with correct node names
coup1_1 = Subsystem(name='coup1_1_sys', sys_type='TL_RESONATOR', nodes=['a1'], q_opts=dict(f_res=7.4, Z0=50, vp=0.404314 * c_light))
coup1_2 = Subsystem(name='coup2_1_sys', sys_type='TL_RESONATOR', nodes=['a2'], q_opts=dict(f_res=7.4, Z0=50, vp=0.404314 * c_light))
coup1_3 = Subsystem(name='coup3_1_sys', sys_type='TL_RESONATOR', nodes=['a3'], q_opts=dict(f_res=7.4, Z0=50, vp=0.404314 * c_light))
coup1_4 = Subsystem(name='coup4_1_sys', sys_type='TL_RESONATOR', nodes=['a4'], q_opts=dict(f_res=7.4, Z0=50, vp=0.404314 * c_light))
coup1_5 = Subsystem(name='coup5_1_sys', sys_type='TL_RESONATOR', nodes=['a5'], q_opts=dict(f_res=7.4, Z0=50, vp=0.404314 * c_light))
coup1_6 = Subsystem(name='coup6_1_sys', sys_type='TL_RESONATOR', nodes=['a6'], q_opts=dict(f_res=7.4, Z0=50, vp=0.404314 * c_light))
coup1_7 = Subsystem(name='coup7_1_sys', sys_type='TL_RESONATOR', nodes=['a7'], q_opts=dict(f_res=7.4, Z0=50, vp=0.404314 * c_light))

coup2_1 = Subsystem(name='coup1_2_sys', sys_type='TL_RESONATOR', nodes=['b1'], q_opts=dict(f_res=7.2, Z0=50, vp=0.404314 * c_light))
coup2_2 = Subsystem(name='coup2_2_sys', sys_type='TL_RESONATOR', nodes=['b2'], q_opts=dict(f_res=7.2, Z0=50, vp=0.404314 * c_light))
coup2_3 = Subsystem(name='coup3_2_sys', sys_type='TL_RESONATOR', nodes=['b3'], q_opts=dict(f_res=7.2, Z0=50, vp=0.404314 * c_light))
coup2_4 = Subsystem(name='coup4_2_sys', sys_type='TL_RESONATOR', nodes=['b4'], q_opts=dict(f_res=7.2, Z0=50, vp=0.404314 * c_light))
coup2_5 = Subsystem(name='coup5_2_sys', sys_type='TL_RESONATOR', nodes=['b5'], q_opts=dict(f_res=7.2, Z0=50, vp=0.404314 * c_light))
coup2_6 = Subsystem(name='coup6_2_sys', sys_type='TL_RESONATOR', nodes=['b6'], q_opts=dict(f_res=7.2, Z0=50, vp=0.404314 * c_light))
coup2_7 = Subsystem(name='coup7_2_sys', sys_type='TL_RESONATOR', nodes=['b7'], q_opts=dict(f_res=7.2, Z0=50, vp=0.404314 * c_light))

readout1 = Subsystem(name='readout1_sys', sys_type='TL_RESONATOR', nodes=['c1'], q_opts=dict(f_res=7.0, Z0=50, vp=0.404314 * c_light))
readout2 = Subsystem(name='readout2_sys', sys_type='TL_RESONATOR', nodes=['c2'], q_opts=dict(f_res=7.0, Z0=50, vp=0.404314 * c_light))
readout3 = Subsystem(name='readout3_sys', sys_type='TL_RESONATOR', nodes=['c3'], q_opts=dict(f_res=7.2, Z0=50, vp=0.404314 * c_light))
readout4 = Subsystem(name='readout4_sys', sys_type='TL_RESONATOR', nodes=['c4'], q_opts=dict(f_res=7.0, Z0=50, vp=0.404314 * c_light))
readout5 = Subsystem(name='readout5_sys', sys_type='TL_RESONATOR', nodes=['c5'], q_opts=dict(f_res=7.0, Z0=50, vp=0.404314 * c_light))
readout6 = Subsystem(name='readout6_sys', sys_type='TL_RESONATOR', nodes=['c6'], q_opts=dict(f_res=7.0, Z0=50, vp=0.404314 * c_light))
readout7 = Subsystem(name='readout7_sys', sys_type='TL_RESONATOR', nodes=['c7'], q_opts=dict(f_res=7.0, Z0=50, vp=0.404314 * c_light))

readout8 = Subsystem(name='readout8_sys', sys_type='TL_RESONATOR', nodes=['d1'], q_opts=dict(f_res=6.8, Z0=50, vp=0.404314 * c_light))
readout9 = Subsystem(name='readout9_sys', sys_type='TL_RESONATOR', nodes=['d2'], q_opts=dict(f_res=6.8, Z0=50, vp=0.404314 * c_light))
readout10 = Subsystem(name='readout10_sys', sys_type='TL_RESONATOR', nodes=['d3'], q_opts=dict(f_res=6.8, Z0=50, vp=0.404314 * c_light))
readout11 = Subsystem(name='readout11_sys', sys_type='TL_RESONATOR', nodes=['d4'], q_opts=dict(f_res=6.8, Z0=50, vp=0.404314 * c_light))
readout12 = Subsystem(name='readout12_sys', sys_type='TL_RESONATOR', nodes=['d5'], q_opts=dict(f_res=6.8, Z0=50, vp=0.404314 * c_light))
readout13 = Subsystem(name='readout13_sys', sys_type='TL_RESONATOR', nodes=['d6'], q_opts=dict(f_res=6.8, Z0=50, vp=0.404314 * c_light))
readout14 = Subsystem(name='readout14_sys', sys_type='TL_RESONATOR', nodes=['d7'], q_opts=dict(f_res=6.8, Z0=50, vp=0.404314 * c_light))

# Composite system with all subsystems
composite_sys = CompositeSystem(
    subsystems=[
        transmon1, transmon2, transmon3, transmon4, transmon5, transmon6, transmon7,
        coup1_1, coup1_2, coup1_3, coup1_4, coup1_5, coup1_6, coup1_7,
        coup2_1, coup2_2, coup2_3, coup2_4, coup2_5, coup2_6, coup2_7,
        readout1, readout2, readout3, readout4, readout5, readout6, readout7,
        readout8, readout9, readout10, readout11, readout12, readout13, readout14
    ],
    cells=[cell_1],
    grd_node='ground_plane',
    nodes_force_keep=[
        'a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7',
        'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7',
        'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7',
        'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7'
    ]
)



In [147]:
cg = composite_sys.circuitGraph()
print(cg)

node_jj_basis:
-------------

['j1', 'Q1_pad_bot', 'j2', 'Q2_pad_bot', 'j3', 'Q3_pad_bot', 'j4', 'Q4_pad_bot', 'j5', 'Q5_pad_bot', 'j6', 'Q6_pad_bot', 'j7', 'Q7_pad_bot', 'a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'P1_Q_launch_pad', 'P2_Q_launch_pad', 'P3_Q_launch_pad', 'P4_Q_launch_pad', 'P5_Q_launch_pad', 'P6_Q_launch_pad', 'P7_Q_ind_stub', 'P7_Q_launch_pad', 'Q1_c_connector_pad', 'Q2_a_connector_pad', 'Q2_c_wire', 'Q3_b_wire', 'Q4_a_wire', 'Q5_a_wire', 'Q6_c_wire', 'Q7_d_wire', 'cpw1_trace', 'cpw3_trace', 'cpw4_trace', 'cpw5_trace', 'cpw6_trace', 'cpw7_trace', 'ol1_trace', 'ol2_trace', 'ol3_trace', 'ol4_trace', 'ol5_trace', 'ol6_trace']

nodes_keep:
-------------

['j1', 'j2', 'j3', 'j4', 'j5', 'j6', 'j7', 'a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'd1', 'd2', 'd3', 'd4', 'd5', 

In [150]:
import numpy as np

# Constants
e = 1.60217663e-19  # Coulombs
h = 6.62607015e-34  # Planck constant, J·s

# Capacitances in femtofarads
capacitances_fF = np.array([412.3, 984.2, 658.2, 2191, 43.10, 107.7, 109.3])

# Convert to farads
capacitances_F = capacitances_fF * 1e-15

# Calculate charging energies in joules
E_C_joules = (e ** 2) / (2 * capacitances_F)

# Convert to GHz: 1 J = 1/h Hz = 1/(h*1e9) GHz
E_C_GHz = E_C_joules / h / 1e9

# Print
for i, ec in enumerate(E_C_GHz):
    print(f"Qubit {i}: {ec:.4f} GHz")



Qubit 0: 0.0470 GHz
Qubit 1: 0.0197 GHz
Qubit 2: 0.0294 GHz
Qubit 3: 0.0088 GHz
Qubit 4: 0.4494 GHz
Qubit 5: 0.1799 GHz
Qubit 6: 0.1772 GHz


In [151]:
import numpy as np

# Constants
phi0 = 2.067833848e-15  # magnetic flux quantum (Wb)
h = 6.62607015e-34      # Planck constant (J·s)

# (Phi0/2pi)^2
phi0_over_2pi_sq = (phi0 / (2 * np.pi))**2

# Diagonal inverse inductance values in 1/nH (convert nH to H)
L_inv_nH = np.array([0.083333, 0.083333, 0.083333, 0.083333, 0.083333, 0.083333, 0.083333])
L_inv_H = L_inv_nH * 1e9  # 1/nH to 1/H (since 1 nH = 1e-9 H, inverse is 1e9)

# Calculate EJ in Joules: EJ = (Phi0/2pi)^2 * L_inv
EJ_J = phi0_over_2pi_sq * L_inv_H

# Convert EJ to frequency in GHz: f = EJ / h (Hz), then to GHz
EJ_GHz = EJ_J / h / 1e9

# Output results
for i, ej in enumerate(EJ_GHz, start=1):
    print(f"Junction j{i}: EJ = {ej:.3f} GHz")


Junction j1: EJ = 13.622 GHz
Junction j2: EJ = 13.622 GHz
Junction j3: EJ = 13.622 GHz
Junction j4: EJ = 13.622 GHz
Junction j5: EJ = 13.622 GHz
Junction j6: EJ = 13.622 GHz
Junction j7: EJ = 13.622 GHz
